```html
<link rel="stylesheet" href="style.css">
```

# Tarea de la Semana 1:<br>Resolución de problemas de conectividad de base de datos en AWS

En este laboratorio, se te asignará la tarea de solucionar y resolver problemas comunes que podrían ocurrir al conectarse a una base de datos. La tarea consta de varios pasos, que incluyen configurar el entorno del laboratorio en AWS, abordar problemas de conectividad desde una instancia EC2 a una instancia de RDS, mejorar las configuraciones de seguridad de RDS y, finalmente, insertar datos en la base de datos y consultarlos. El laboratorio te guía a través de la depuración de los problemas de conectividad mientras te brinda experiencia práctica con los servicios de AWS y SQL.

# Tabla de Contenidos
- [1 - Solución de Problemas de Conectividad de Base de Datos](#1)
- [2 - Solución de Problemas de Permisos](#2)

---


<div id='1'/>

## 1 - Solucionando Problemas de Conectividad con la Base de Datos


Se le proporciona una instancia de RDS a la que intentará conectarse desde una instancia de EC2.

1.1. Para acceder a la consola de AWS, ejecute el siguiente comando en la terminal.
Si es necesario, abra una nueva terminal seleccionando Terminal > Nueva terminal en el menú.

```bash
cat ../.aws/aws_console_url
```
Abra el enlace en una nueva ventana del navegador.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS caducará cada 15 minutos, 
pero cualquier recurso de AWS que haya creado seguirá disponible durante el período de 2 horas. 
Si necesita acceder a la consola después de 15 minutos, vuelva a ejecutar el comando para obtener un nuevo enlace activo.

*Nota:* Si ve la ventana como en la siguiente captura de pantalla, haga clic en el enlace **cerrar sesión**, 
cierre la ventana y vuelva a hacer clic en el enlace de la consola.

![AWSLogout](images/AWSLogout.png)


1.2. Ve a **RDS** en la consola de AWS y haz clic en la instancia de base de datos `de-c2w1a1-rds` para ver los detalles. Verifica el Endpoint y el Puerto de RDS, puedes copiar esos valores y guardarlos localmente.

![imagen alt ><](./images/RDS.png)

1.3. Ve a **EC2**. En el panel izquierdo, haz clic en **Instancias**.

![imagen alt ><](./images/EC2.png)

1.4. Verás que ya se te han proporcionado dos instancias de EC2. Por el momento, trabajarás solo con la que se llama `de-c2w1a1-external-bastion-host`.

- Haz clic en el **ID de la instancia** para `de-c2w1a1-external-bastion-host`.
- En el panel superior, haz clic en **Conectar**.

![imagen alt ><](./images/EC2_connect.png)

- En la pestaña *Conexión de Instancia EC2*, haz clic en **Conectar**. Se te pedirá que accedas al terminal de la instancia EC2.

![imagen alt ><](./images/EC2_connect2.png)


1.5. Para conectarse a la instancia de RDS, el equipo de DevOps te proporcionó las siguientes credenciales para acceder a la base de datos:

- nombre de usuario: `postgres`
- contraseña: `postgrespwrd`

La base de datos es una instancia de PostgreSQL, por lo que primero necesitas instalar el cliente `psql`; `psql` te permite establecer una conexión con la base de datos PostgreSQL.
Ejecuta el siguiente comando para instalar el cliente `psql` en la instancia de Amazon Linux 2:

```bash 
sudo amazon-linux-extras install postgresql10 -y
```

*Nota*: En la terminal de EC2, dependiendo de tu navegador y máquina, es posible que los atajos regulares para copiar (`Ctrl+C` o `Cmd+C`) y pegar (`Ctrl+V` o `Cmd+V`) no funcionen. En su lugar, puedes utilizar
`Ctrl + Insert` para copiar y `Shift + Insert` para pegar o

`clic derecho` -> `copiar`,
`clic derecho` -> `pegar`.


```markdown
# Welcome to my Markdown Translator

## Instructions:
1. Copy the text you want to translate.
2. Paste it here.
3. Get the translated text in Markdown format!

### Let's get started!
```


1.6. Ahora necesitas obtener el punto de conexión de la base de datos. Puedes usar el punto de conexión que guardaste localmente en el paso 1.2, o puedes obtener el punto de conexión ejecutando el siguiente comando, reemplazando `<PostgreSQL-DB-name>` con `de-c2w1a1-rds`:

``` bash
aws rds describe-db-instances --db-instance-identifier <PostgreSQL-DB-name> --output text --query "DBInstances[].Endpoint.Address" --region us-east-1
```


1.7. Intenta conectarte a tu base de datos RDS ejecutando el siguiente comando. Reemplaza `<PostgreSQLEndpoint>` con la salida del paso anterior. El puerto 5432 es el predeterminado para PostgreSQL, y el nombre de usuario y la contraseña te son proporcionados por el equipo de DevOps.

*Nota*: El siguiente comando te pedirá que ingreses la contraseña de la base de datos. Cuando escribas o pegues la contraseña, no se mostrará nada en la pantalla, esto es una característica de seguridad común en varios programas de terminal.

```bash
psql --host=<PostgreSQLEndpoint> --username=postgres --password --port=5432
```

Presionar enter te mostrará un error o líneas vacías... la conexión falla.

![image alt ><](./images/psqlConnect.png)

Para salir del intento presiona `Ctrl+C` (también podría ser `Cmd+C` o `Ctrl+S`).


1.8. Entonces.... la conexión a la base de datos falló. ¿Cómo puedes solucionar el problema? ¿Por dónde deberías empezar? Dado lo que sabes sobre redes, ¿puedes pensar en algún problema que pueda estar surgiendo de la configuración de red que tienes para la instancia EC2 y la instancia de la base de datos?

La instancia de base de datos RDS reside dentro de un VPC (Virtual Private Cloud), y la instancia EC2, desde la cual intentaste la conexión, también se lanzó dentro de un VPC. Ahora, si la base de datos RDS y la instancia EC2 no se lanzaron dentro del mismo VPC y los dos VPC no están configurados para comunicarse entre sí, entonces esta podría ser una razón por la cual la conexión falló. Verifiquemos la configuración del VPC de cada uno.

Ve a **RDS** en la consola de AWS, haz clic en **Bases de datos** y luego haz clic en la instancia de base de datos `de-c2w1a1-rds`. Verifica el VPC en **Conectividad y seguridad**. Haz clic en el identificador de **VPC** y encuentra el **ID de VPC** correspondiente. Haz clic en el **ID de VPC** y luego verifica el mapa de recursos asociado con esta configuración de red.


1.9. Abre otra pestaña e ve a **EC2** en la consola de AWS. Haz clic en **Instancias** y luego haz clic en el **ID de la instancia** de la instancia llamada `de-c2w1a1-external-bastion-host`. Luego, desplázate hacia abajo y busca la pestaña de **Networking** y haz clic en ella.

![image alt ><](./images/EC2NetworkSettings.png)

Verifica el identificador de **VPC** (este es el ID de VPC) y puedes hacer clic en él para ver el mapa de recursos asociado. Te darás cuenta de que las instancias de RDS y EC2 se lanzaron en VPCs diferentes, por eso no pueden comunicarse: la instancia de EC2 se lanzó en un VPC llamado `de-c2w1a1-external-bastion-host-vpc` mientras que la instancia de RDS se lanzó en otro VPC llamado `de-c2w1a1`.


1.10. Una solución podría ser utilizar una instancia EC2 implementada en la misma VPC que la instancia de RDS. Sin embargo, en este laboratorio, no necesitas hacer eso porque ya se te proporcionó una segunda instancia llamada `de-c2w1a1-bastion-host`, que está implementada en la VPC correcta.

En la consola de AWS, ve a **EC2**, y busca el **ID de la instancia** para la que se llama `de-c2w1a1-bastion-host`.

*Opcional*: Puedes verificar ahora que esta instancia EC2 está en la misma VPC que tu base de datos (ver pasos 1.8-1.9).


1.11. Conéctese nuevamente a la nueva instancia de EC2 y pruebe la conexión a la base de datos (repita los pasos 1.5-1.7).

Verá que la conexión aún no funciona. Si tiene la instancia de EC2 y la RDS lanzadas en la misma VPC, ¿por qué aún no puede conectarse a la base de datos? ¿Está configurada la instancia de RDS para recibir tráfico de la instancia de EC2? ¿Cómo puede verificarlo?

Entonces, la segunda cosa que puede verificar es el grupo de seguridad de la instancia de RDS, para verificar qué tráfico entrante se permite llegar a la instancia de RDS. ¡Vamos a verificarlo!


1.12. En la consola de AWS, busca **RDS**, ve a **Bases de datos** y luego haz clic en la instancia de base de datos `de-c2w1a1-rds`. En la sección de **Conectividad y seguridad**, haz clic en el enlace de **Grupos de seguridad de VPC**. Luego haz clic en el **ID del grupo de seguridad** para que puedas verificar las reglas de entrada.

Verás solo una regla de entrada, que permite `Todo el tráfico` pero desde el mismo grupo de seguridad. Esto significa que cualquier instancia asociada al mismo grupo de seguridad puede comunicarse libremente con otras instancias en el mismo grupo de seguridad. El propósito de esta regla predeterminada es facilitar la comunicación entre recursos dentro del mismo grupo de seguridad mientras se mantiene un límite seguro con respecto a los recursos fuera del grupo. Es una forma conveniente de permitir la comunicación entre instancias dentro de un contexto específico, como dentro de una capa de aplicación o a través de diferentes componentes de un sistema distribuido.


1.13. Sabes que la base de datos RDS es una base de datos PostgreSQL, y el puerto predeterminado permitido para establecer comunicación con una base de datos PostgreSQL es el puerto `5432`. Necesitas agregar una nueva regla que permita el tráfico a ese puerto. Haz clic en el botón **Editar reglas de entrada**, luego en **Agregar regla**. Para el tipo, utiliza `TCP personalizado`, y para el rango de puertos, simplemente usa el número de puerto `5432`. Para la fuente, puedes elegir `0.0.0.0/0` de la lista de búsqueda de *Origen*.

![inbound_rule.png](images/InboundRule1.png)

Sin embargo, agregar `0.0.0.0/0` como fuente significa permitir que cualquier tráfico público se conecte a la base de datos. Esta no es la forma más segura, pero podría ser una opción buena y rápida para probar la conexión. La mejor práctica es permitir solo a los recursos necesarios conectarse a la base de datos. En este momento, solo la instancia EC2 necesita acceder a la base de datos. Así que asegurémonos de aplicar directamente la mejor práctica en este ejercicio.

- En otra pestaña del navegador, busca **EC2** en la consola de AWS. Haz clic en **Instancias** y luego en el **ID de instancia** para `de-c2w1a1-bastion-host`. Ve a la pestaña **Seguridad** y haz clic en el enlace **Grupo de seguridad**. Copia el **ID del grupo de seguridad**.

- Ahora vuelve a la pestaña donde estabas editando la regla de entrada para RDS. En el campo *Origen*, pega el ID del grupo de seguridad que acabas de copiar y luego haz clic en el Grupo de Seguridad que aparece en el menú desplegable.

![inbound_rule.png](images/InboundRule2.png)

Haz clic en **Guardar reglas**.


1.14. Regresa a la instancia de EC2 y conéctate nuevamente a la base de datos (repite los pasos 1.5-1.7). Ahora obtendrás un error similar al siguiente:

```bash
psql: FATAL:  la autenticación de contraseña falló para el usuario "postgres"
FATAL:  no hay entrada pg_hba.conf para el host "XX.X.X.XXX", usuario "postgres", base de datos "postgres", sin encriptación
```

Este error significa que la contraseña proporcionada es incorrecta. Supongamos que hablaste con el equipo de DevOps: resultó que tuvieron que realizar un mantenimiento y actualizaron tu contraseña a `adminpwrd`.


1.15. ¡Intenta conectarte nuevamente a la base de datos usando la nueva contraseña, debería funcionar ahora! Ingresa `\q` en la terminal para cerrar la conexión.

¡Trabajo fantástico! Después de mucho tiempo depurando tu conexión RDS, ¡finalmente lo lograste!


<div id='2'/>

## 2 - Solucionando Problemas de Permisos

En esta parte del laboratorio, ejecutarás algunas declaraciones SQL para crear una tabla dentro de la base de datos RDS y poblarla con algunos datos de un bucket S3.

2.1. Ejecuta el siguiente comando en la terminal de EC2 para descargar los archivos requeridos para el laboratorio.

*Nota*: Asegúrate de ejecutar todos los comandos de esta sección en la terminal de EC2, no en la terminal de VSCode.

```bash
aws s3 cp --recursive s3://dlai-data-engineering/labs/c2w1a1-814493-vocapi/ ./
```


2.2. Ahora creará una tabla en la base de datos. Abra el archivo `sql/ratings_table_ddl.sql` en la terminal de EC2 con `nano -cv sql/ratings_table_ddl.sql` para inspeccionarlo sin modificarlo. Ahora que tiene algo de experiencia con SQL, intente comprender el código proporcionado. Para cerrar el archivo, simplemente presione `Ctrl + x` o `Cmd + x`.


2.3. Conéctese nuevamente a la base de datos (paso 1.7) y ejecute el archivo SQL con el siguiente comando:

```bash
\i sql/ratings_table_ddl.sql
```

**Salida esperada**:
```text
postgres=> \i sql/ratings_table_ddl.sql
psql:sql/ratings_table_ddl.sql:1: AVISO: la tabla "ratings_training" no existe, se omite
DROP TABLE
```


2.4. Ejecuta la consulta para verificar que la tabla está actualmente vacía:

```sql
SELECT * FROM ratings_training;
```


2.5. Sal de la conexión a la base de datos con el comando `\q`.

2.6. Los datos que necesitas ingresar en la nueva tabla de la base de datos existen en un bucket de S3 que se proporciona en este laboratorio. Puedes encontrar el nombre del bucket de S3 en la consola de AWS, o ejecutando uno de los siguientes comandos `aws s3 ls` o

```bash
aws s3api list-buckets --output text --query "Buckets[].Name"
```

Guarda el resultado localmente.

Descarguemos los datos del bucket de S3 en un directorio dentro de la instancia EC2. Crea el directorio `data`:

```bash
mkdir -p data
```


2.7. Para descargar el archivo del bucket S3, utilizarás el script de Python ubicado en `scripts/download_from_s3.py`. Abre el archivo y revisa su contenido. En la terminal de EC2, abre el archivo con `nano -c scripts/download_from_s3.py`. Reemplaza el marcador de posición `<YOUR-DATA-BUCKET>` con el nombre del bucket S3 que encontraste en el paso anterior. Una vez que hayas modificado el archivo, presiona `Ctrl + o` o `Cmd + o`, y luego presiona `Enter` para guardar el archivo. Ciérralo con `Ctrl + x` o `Cmd + x`.

2.8. Para ejecutar el script, necesitarás instalar `boto3`:

```bash
pip3 install boto3
```


2.9. Ahora intenta ejecutar el código de Python con el siguiente comando:

```bash
python3 scripts/download_from_s3.py
```

Obtendrás un error similar a este:

```bash
Error al descargar el archivo: Se produjo un error (403) al llamar a la operación HeadObject: Prohibido
```

Este mensaje de error significa que no tienes permiso para descargar datos del bucket S3.


2.10. Para resolver tu problema, el equipo de DevOps te permitió agregar una política de bucket. Esta política debe estar restringida únicamente a tu entorno de desarrollo en la instancia EC2 y solo para leer desde la carpeta donde se almacenan los datos en el bucket (el nombre de la carpeta es `csv`).

Ve a la consola de AWS, busca **S3**, y haz clic en el nombre de tu bucket. Luego, ve a la pestaña de **Permisos** y desplázate hacia abajo hasta encontrar la sección de Política de Bucket. Haz clic en **Editar**.


2.11. Elimine la política proporcionada y cámbiela por la siguiente política:

```json
{
    "Version": "2012-10-17",
    "Statement": [
         {
            "Effect": "Allow",
            "Principal": "*",
            "Action": "s3:GetObject",
            "Resource": "arn:aws:s3:::<NOMBRE-DE-SU-BUCKET>/csv/*",
            "Condition": {
                "IpAddress": {
                    "aws:SourceIp": "<DIRECCIÓN-IP-DE-SU-HOST-BASTIÓN>" 
                }
            }
         }
     ]
}
```

Reemplace los marcadores de posición:
- `<NOMBRE-DE-SU-BUCKET>`: nombre de su bucket de S3 (consulte el paso 2.6),
- `<DIRECCIÓN-IP-PÚBLICA-DE-SU-HOST-BASTIÓN>`: la dirección IP pública de la instancia EC2 en la que está trabajando. Encuentre **EC2** en la consola de AWS, haga clic en **Instancias**, y luego en el enlace **ID de instancia** para el que se llama `de-c2w1a1-bastion-host`. Encuentre y copie la **dirección IPv4 pública** de la sección **Resumen de la instancia**.

Revise la política e intente comprenderla (para obtener más información, puede consultar la [documentación](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference_policies_elements_condition_operators.html#Conditions_IPAddress). Esta es una política basada en recursos, asegúrese de consultar los materiales de lectura de la lección 2 para aprender más sobre este tipo de política):

* `"Effect": "Allow"`: Esto significa que permite que se realicen ciertas acciones sobre algunos recursos.
* `"Action": "s3:GetObject"`: Esto especifica que el método `get_object` está permitido para realizarse en el bucket de S3 para obtener objetos de él.

Asegúrese de guardar los cambios que ha realizado en la política (haga clic en **Guardar cambios**).


2.12. Repita el paso 2.9 para descargar los datos del bucket S3 en la carpeta `data` que creó.

2.13. Explore el archivo `data/ratings_ml_training_dataset.csv` para entender la estructura de los datos. Puede hacerlo con el siguiente comando:

```bash
head data/ratings_ml_training_dataset.csv
```


2.14. Abre el archivo `sql/copy_data.sql` y explora el código. Recuerda usar el comando `nano -cv sql/copy_data.sql` y cierra el archivo con `Ctrl + x` o `Cmd + x`.


2.15. Luego, conectarse a la base de datos nuevamente (paso 1.7) y poblar la base de datos usando el siguiente comando:
    
```bash
\i sql/copy_data.sql
```


2.16. Después de este proceso, deberías poder consultar tu tabla. Puedes utilizar el conocimiento que adquiriste en el laboratorio del curso anterior para explorar los datos. Como punto de partida, puedes utilizar la siguiente consulta y también intentar contar el número de filas que se han insertado.

```sql
SELECT * FROM ratings_training LIMIT 10;
```
